In [6]:
# input_path = "documentacion/1. Estatutos (2022).md"
# input_path = "documentacion/2. Programa Educativo de Grupo 2022 - 2025.md"
input_path = "documentacion/3. Reglamento de Regimen Interno (2023).md"
output_path = "vector_db"

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [8]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

## MARKDOWN LOADER

In [9]:
def read_md_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            md_content = file.read()
        return md_content
    except FileNotFoundError:
        print("El archivo no fue encontrado.")
        return None

# Ejemplo de uso
md_string = read_md_file(input_path)
md_string

'# Reglamento de Regimen Interno (2023)\n\n\n\n## CAPÍTULO I. ASOCIACIÓN ÍNDICE\n\n#### Artículo 1.- Definición:...........................................................................................................\n\n```\nLa Asociación Grupo Scout Annapurna (GSA), en adelante el Grupo, se define como\nun conjunto de personas que trabaja, aprende y forma en el escultismo. Es una entidad\nautogestionada, no lucrativa y de carácter social, apartidista, aconfesional y dedicada\na la labor educativa de jóvenes de edades comprendidas entre 6 y 21 años inspirada\nen fines, principios y métodos del movimiento scout, fundado por Lord Baden-Powell.\n```\n#### Artículo 2.- Régimen Jurídico\n\n```\nEl Grupo se constituye como una entidad sin ánimo de lucro, al amparo del artículo 22\nde la Constitución Española (CE), que se rige por la Ley Orgánica 1/2002, de 22 de\nmarzo reguladora del derecho de Asociación y normas concordantes, así como\naquellas disposiciones futuras que en materia asoci

In [10]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

# headers_to_split_on = [
#     ("#", "source"),
#     ("##", "capitulo"),
#     ("###", "apartado"),
#     ("####", "subapartado"),
# ]
headers_to_split_on = [
    ("#", "source"),
    ("##", "capitulo"),
    ("###", "seccion"),
    ("####", "articulo"),
]


markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
splits = markdown_splitter.split_text(md_string)
for doc in splits:
    print(doc.metadata)

{'source': 'Reglamento de Regimen Interno (2023)', 'capitulo': 'CAPÍTULO I. ASOCIACIÓN ÍNDICE', 'articulo': 'Artículo 1.- Definición:...........................................................................................................'}
{'source': 'Reglamento de Regimen Interno (2023)', 'capitulo': 'CAPÍTULO I. ASOCIACIÓN ÍNDICE', 'articulo': 'Artículo 2.- Régimen Jurídico'}
{'source': 'Reglamento de Regimen Interno (2023)', 'capitulo': 'CAPÍTULO I. ASOCIACIÓN ÍNDICE', 'articulo': 'Artículo 3.- Objetivos y fines'}
{'source': 'Reglamento de Regimen Interno (2023)', 'capitulo': 'CAPÍTULO I. ASOCIACIÓN ÍNDICE', 'articulo': 'Artículo 4.- Actividades'}
{'source': 'Reglamento de Regimen Interno (2023)', 'capitulo': 'CAPÍTULO I. ASOCIACIÓN ÍNDICE', 'articulo': 'Artículo 5.- Colaboración con otras entidades.'}
{'source': 'Reglamento de Regimen Interno (2023)', 'capitulo': 'CAPÍTULO I. ASOCIACIÓN ÍNDICE', 'articulo': 'Artículo 6.- Principios'}
{'source': 'Reglamento de Regimen Interno (20

## VECTORS

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
vectordb = Chroma(persist_directory=output_path, embedding_function=HuggingFaceEmbeddings())
print(vectordb._collection.count())

77


In [13]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=HuggingFaceEmbeddings(),
    persist_directory=output_path
)

In [14]:
print(vectordb._collection.count())

154


## CLASE

In [ ]:
# input_path = "pipeline_files/2_transcribed_audio/gestion_de_requisitos_con_Redmine.json"
input_path = "pipeline_files/2_transcribed_audio_coffee"
# output_path = "pipeline_files/3_vectordb"
output_path = "pipeline_files/3_vectordb_coffee"
chunk_duration = 10 # in seconds
reset_db = True

In [ ]:
import json
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os
import shutil
import torch


class VectorDBGenerator:
    def __init__(self, output_path, reset_db=True):
        self.output_path = output_path
        if reset_db:
            self.__delete_db()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.embeddings = HuggingFaceEmbeddings()

    def generate_vectordb(self, input_path, chunk_duration):
        json_data = self.__read_json_file(input_path)
        chunks = self.__chunk_aggregator(json_data, chunk_duration)
        documents = self.__generate_documents(chunks)
        vectordb = self.__generate_vectors(documents)
        return vectordb
    

    def __read_json_file(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data


    def __chunk_aggregator(self, data, chunk_duration):
        # duration_chunk, chunk_text, chunk_start, chunk_end = 0, "", "00:00:00,000", "00:00:00,000"
        duration_chunk, chunk_text, chunk_start, chunk_end = 0, "", "00:00:00.000", "00:00:00.000"
        chunks = []
        for el in data:
            start = float(self.__time_to_seconds(el["start"]))
            end = float(self.__time_to_seconds(el["end"]))
            duration_chunk += end - start
            # chunk_text += el["text"] + " "
            chunk_text += el["content"] + " "
            if duration_chunk >= chunk_duration:
                chunk_end = el["end"]
                # chunks.append({"filename": el["filename"], "start": chunk_start, "end": chunk_end, "text": chunk_text})
                chunks.append({"episode": el["episode"], "start": chunk_start, "end": chunk_end, "content": chunk_text})
                chunk_start = el["end"]
                chunk_text = ""
                duration_chunk = 0
        return chunks
    
    def __time_to_seconds(self, time_str):
        time_components = time_str.split(':')
        hours = int(time_components[0])
        minutes = int(time_components[1])
        # seconds, milliseconds = map(float, time_components[2].split(','))
        seconds, milliseconds = map(float, time_components[2].split('.'))
        total_seconds = hours * 3600 + minutes * 60 + seconds + milliseconds / 1000
        return total_seconds
    

    def __generate_documents(self, data):
        splits = []
        for el in data:
            metadata = {}
            # metadata["filename"] = el["filename"]
            # metadata["start"] = el["start"]
            # metadata["end"] = el["end"]
            # doc =  Document(page_content=el["text"], metadata=metadata)
            metadata["filename"] = el["episode"]
            metadata["start"] = el["start"]
            metadata["end"] = el["end"]
            doc =  Document(page_content=el["content"], metadata=metadata)
            splits.append(doc)
        return splits
    
    def __generate_vectors(self, documents):
        vectordb = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings,
            persist_directory=self.output_path
        )
        return vectordb
    
    def __delete_db(self):
        if os.path.exists(self.output_path):
            shutil.rmtree(self.output_path)
        os.makedirs(self.output_path)


In [ ]:
# generator = VectorDBGenerator(output_path, reset_db)
# if os.path.isdir(input_path):
#     for file_name in os.listdir(input_path):
#         if file_name.endswith(".json"):
#             file_path = os.path.join(input_path, file_name)
#             generator.generate_vectordb(file_path, chunk_duration)
# else:
#     generator.generate_vectordb(input_path, chunk_duration)

In [ ]:
from tqdm import tqdm

generator = VectorDBGenerator(output_path, reset_db)
if os.path.isdir(input_path):
    for file_name in tqdm(os.listdir(input_path), desc="Processing files"):
        if file_name.endswith(".json"):
            file_path = os.path.join(input_path, file_name)
            generator.generate_vectordb(file_path, chunk_duration)
else:
    generator.generate_vectordb(input_path, chunk_duration)
